# CHAPTER5 텐서보드와 모델 재사용

- 학습 모델 저장하고 재사용하기
- 텐서보드 사용하기

---

#### 5.1 학습 모델 저장하고 재사용하기

In [1]:
# import library
import tensorflow as tf
import numpy as np

In [2]:
# load data
data = np.loadtxt('data/data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [3]:
# make count
global_step = tf.Variable(0, trainable=False, name='global_step')

# hypothesis
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))

# make model
model = tf.matmul(L2, W3)

In [4]:
# cost function
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

In [5]:
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

In [6]:
# sess run
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

In [7]:
# check model
ckpt = tf.train.get_checkpoint_state('model/model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [8]:
# optimizer
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step: 1,  Cost: 1.729
Step: 2,  Cost: 1.597


In [9]:
# store checkpoint
saver.save(sess, 'model/dnn.ckpt', global_step=global_step)

'model/dnn.ckpt-2'

In [10]:
# prediction
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [0 2 0 0 0 0]
실제값: [0 1 2 0 0 2]


In [11]:
# accuracy
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도: 50.00


---

#### 5.2 텐서보드 사용하기

In [12]:
# make count
global_step = tf.Variable(0, trainable=False, name='global_step')

# hypothesis
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

    tf.summary.scalar('cost', cost)

In [13]:
# sess run
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('model/model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [14]:
# collect tensor
merged = tf.summary.merge_all()

In [15]:
# store
writer = tf.summary.FileWriter('./logs', sess.graph)

In [16]:
# optimizer
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step+1) % 10 == 0 :
        
        print('Step: %d, ' % sess.run(global_step),
              'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    # 적절한 시점에 저장할 값들을 수집하고 저장합니다.
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

Step: 10,  Cost: 0.762
Step: 20,  Cost: 0.587
Step: 30,  Cost: 0.559
Step: 40,  Cost: 0.553
Step: 50,  Cost: 0.551
Step: 60,  Cost: 0.551
Step: 70,  Cost: 0.550
Step: 80,  Cost: 0.550
Step: 90,  Cost: 0.550
Step: 100,  Cost: 0.550


In [17]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-100'

In [18]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]


In [19]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도: 100.00
